In [1]:
import numpy as np
import cv2

# Lucas Kanade Method

In [14]:
corner_track_parms = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

In [15]:
# LUCAS-KANADE METHOD PARAMETERS
# winSize is a trade off bcoz small win size is used to detect small motions and for large motions large win size is required.
# maxLevel is related to image pyramid more the level is lower the resolution by 1/2^level
# 10 is count and 0.03 is EPSILON

lk_params = dict(winSize=(200,200), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 50, 0.032))

In [20]:
cap=cv2.VideoCapture('Input.mp4')

ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# POINTS TO TRACK
prevPoints = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_parms)

# Mask is for drawing lines on the video then tracking point 
mask = np.zeros_like(prev_frame)

while True:
    
    ret, frame = cap.read()
    if ret:
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        nextPoints, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPoints, None, **lk_params)

    #     status = 1 is features and points found
        good_new = nextPoints[status==1]
        good_prev = prevPoints[status==1]

        for i, (new, prev) in enumerate(zip(good_new, good_prev)):

            x_new, y_new = new.ravel()
            x_prev, y_prev = prev.ravel()

            mask = cv2.line(mask, (x_new, y_new), (x_prev, y_prev), (0,255,0), 3)

            frame = cv2.circle(frame, (x_new, y_new), 8, (0,0,255), -1)

        img = cv2.add(frame,mask)

        cv2.imshow("Tracking", img)

        k = cv2.waitKey(30) & 0xFF
        if k == 27:
            break

        prev_gray = frame_gray
        prevPoints = good_new.reshape(-1,1,2)
    else:
        break
    
    
cv2.destroyAllWindows()
cap.release()

# DENSE POINT MAPPING

In [21]:
cap=cv2.VideoCapture('Input.mp4')

ret, frame1 = cap.read()

prevFrame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    ret, frame2 = cap.read()
    
    if ret:
        nextFrame = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        flow = cv2.calcOpticalFlowFarneback(prevFrame, nextFrame,None, 0.5, 3, 15, 3, 5, 1.2, 0)

    #   Converting flow into polar coordinates where magnitude = saturation, angle = Hue
        mag, angle = cv2.cartToPolar(flow[:,:,0], flow[:,:,1], angleInDegrees=True)

        hsv_mask[:,:,0] = angle/2   #360 deg to 180 deg
        hsv_mask[:,:,2] = cv2.normalize(mag, None,0, 255, cv2.NORM_MINMAX)

        bgr = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
        cv2.imshow("Frame", bgr)

        k = cv2.waitKey(30) & 0xFF
        if k == 27:
            break

        prevFrame = nextFrame
    else:
        break
    
cv2.destroyAllWindows()
cap.release()